# **Chapter 12: Custom Models and Training with TensorFlow**

## **1. Pendahuluan**

[cite_start]Hingga saat ini, kita telah menggunakan API tingkat tinggi TensorFlow, yaitu `tf.keras`, yang sangat memudahkan pembuatan berbagai arsitektur jaringan saraf. Faktanya, 95% kasus penggunaan yang akan Anda temui dapat diselesaikan hanya dengan `tf.keras`[cite: 1850].

Namun, ada kalanya kita perlu menyelam lebih dalam ke API tingkat rendah TensorFlow. Hal ini diperlukan ketika kita membutuhkan kontrol ekstra untuk menulis:
* Fungsi *loss* kustom.
* Metrik, layer, atau model kustom.
* Loop pelatihan (*training loop*) manual yang kompleks (misalnya untuk menerapkan transformasi gradien khusus).

[cite_start]Bab ini akan membahas cara menggunakan API tingkat rendah Python TensorFlow untuk menangani kasus-kasus tersebut, serta bagaimana meningkatkan performa model kustom menggunakan fitur pembuatan grafik otomatis (*automatic graph generation*)[cite: 1853, 1854].


Mari kita mulai dengan tur singkat mengenai dasar-dasar TensorFlow.

## **2. Menggunakan TensorFlow seperti NumPy**

API TensorFlow berpusat pada **tensor**, yang mengalir dari satu operasi ke operasi lainnya. [cite_start]Tensor sangat mirip dengan `ndarray` pada NumPy, biasanya berupa array multidimensi, tetapi juga bisa menampung skalar[cite: 1964, 1965].

### **Tensor dan Operasi**
Kita dapat membuat tensor menggunakan `tf.constant()`. TensorFlow menyediakan berbagai operasi matematika dasar yang mirip dengan NumPy, seperti `tf.add`, `tf.multiply`, `tf.square`, dll.

In [ ]:
import tensorflow as tf
import numpy as np

# Membuat tensor matriks
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

print("Tensor t:\n", t)
print("Shape:", t.shape)
print("Dtype:", t.dtype)

# Operasi Tensor
# Penjumlahan dan pengkuadratan
print("\nt + 10:\n", t + 10)
print("Square(t):\n", tf.square(t))

# Perkalian matriks (@ operator)
print("t @ t.T:\n", t @ tf.transpose(t))

### **Variabel**
Tensor biasa bersifat *immutable* (tidak dapat diubah). [cite_start]Untuk parameter model (seperti bobot dan bias) yang perlu diperbarui selama *backpropagation*, kita membutuhkan **`tf.Variable`**[cite: 2066, 2069].

Variabel dapat dimodifikasi *in-place* menggunakan metode `assign()`, `assign_add()`, atau `assign_sub()`.

In [ ]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

# Mengubah nilai variabel
v.assign(2 * v)           # Mengalikan nilai dengan 2
v[0, 1].assign(42)        # Mengubah elemen spesifik

print("Variabel v yang telah dimodifikasi:\n", v.numpy())

## **3. Custom Loss Functions**

Terkadang fungsi *loss* standar tidak cukup untuk kebutuhan spesifik kita. Misalnya, untuk data yang *noisy* (berisik), *Mean Squared Error* (MSE) mungkin terlalu sensitif terhadap outlier, sedangkan *Mean Absolute Error* (MAE) mungkin kurang presisi saat konvergensi. [cite_start]**Huber Loss** adalah solusi di antara keduanya[cite: 2116, 2117, 2118].

Mari kita implementasikan Huber Loss secara manual. Fungsi ini harus menerima label asli (`y_true`) dan prediksi (`y_pred`), lalu mengembalikan nilai *loss* per instance.

In [ ]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Penggunaan dalam model Keras
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1, input_shape=[8])])
model.compile(loss=huber_fn, optimizer="nadam")
print("Model berhasil dikompilasi dengan custom loss function.")

### **Menyimpan Model dengan Komponen Kustom**
Saat memuat model yang memiliki komponen kustom, kita perlu memetakan nama fungsi ke fungsi aslinya. [cite_start]Jika *loss function* memiliki *hyperparameter* (misalnya `threshold` pada Huber Loss), lebih baik mengimplementasikannya sebagai *subclass* dari `keras.losses.Loss` agar konfigurasinya ikut tersimpan[cite: 2162, 2164].

In [ ]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
        
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# Penggunaan kelas custom loss
model.compile(loss=HuberLoss(2.0), optimizer="nadam")

## **4. Custom Metrics**

Metrik digunakan untuk evaluasi dan harus mudah diinterpretasikan manusia. Tidak seperti *loss*, metrik tidak perlu *differentiable*.

Untuk metrik sederhana, kita bisa menggunakan fungsi seperti *loss*. [cite_start]Namun, untuk metrik yang bersifat *streaming* (dihitung secara bertahap per *batch*, seperti **Precision**), kita perlu membuat *subclass* dari `keras.metrics.Metric`[cite: 2264, 2271].

Objek *streaming metric* menjaga status internal (variabel) yang diperbarui setiap *batch* melalui metode `update_state()`.

In [ ]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        # Variabel status
        self.huber_fn = lambda y_true, y_pred: HuberLoss(threshold).call(y_true, y_pred)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
        
    def result(self):
        return self.total / self.count
        
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

## **5. Custom Layers**

Jika Anda membutuhkan layer yang tidak tersedia di Keras, Anda bisa membuatnya sendiri.
* [cite_start]**Layer tanpa bobot (Stateless):** Gunakan `keras.layers.Lambda`[cite: 2311].
* [cite_start]**Layer dengan bobot (Stateful):** Buat *subclass* dari `keras.layers.Layer`[cite: 2317].

Layer kustom harus mengimplementasikan:
1.  [cite_start]`build()`: Membuat bobot layer (dipanggil saat *shape* input diketahui)[cite: 2343].
2.  [cite_start]`call()`: Melakukan operasi perhitungan (forward pass)[cite: 2353].
3.  [cite_start]`compute_output_shape()`: Menentukan bentuk output[cite: 2354].

In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        # Membuat bobot kernel dan bias
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        # Operasi: X * W + b
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

## **6. Custom Models**

Untuk arsitektur yang kompleks (misalnya dengan *skip connections* atau loop), kita bisa membuat *subclass* dari `keras.Model`. [cite_start]Ini mirip dengan custom layer, tetapi dengan fungsi tambahan seperti `compile()`, `fit()`, dan `save()`[cite: 2385, 2386].

Contoh implementasi model dengan **Residual Block** (seperti pada ResNet):

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="elu",
                                             kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # Skip connection: menambahkan input asli ke output

class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation="elu",
                                             kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

### **Losses Berdasarkan Internal Model**
Terkadang kita ingin menghitung *loss* berdasarkan bagian internal model, bukan hanya outputnya. Contohnya adalah **Reconstruction Loss** pada *autoencoder* untuk regularisasi. [cite_start]Kita bisa menggunakan metode `add_loss()` di dalam metode `call()`[cite: 2445, 2448].

## **7. Menghitung Gradien dengan Autodiff**

Untuk memahami cara kerja *training loop* kustom, kita perlu memahami **Automatic Differentiation (Autodiff)**. [cite_start]TensorFlow menggunakan `tf.GradientTape` untuk merekam operasi agar gradien dapat dihitung secara otomatis[cite: 2516, 2519].

Tape akan merekam operasi yang melibatkan variabel, lalu kita bisa memanggil `tape.gradient()` untuk mendapatkan turunannya.

In [ ]:
def f(w1, w2):
    return 3 * w1**2 + 2 * w1 * w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)

with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print("Gradien:", gradients) 
# Turunan thd w1: 6*w1 + 2*w2 = 30 + 6 = 36
# Turunan thd w2: 2*w1 = 10

## **8. Custom Training Loops**

Meskipun metode `fit()` sangat kuat, ada kalanya kita membutuhkan kendali penuh atas proses pelatihan (misalnya menggunakan dua optimizer berbeda). Kita bisa menulis loop pelatihan sendiri.

Langkah-langkah utamanya:
1.  Iterasi dataset per *batch*.
2.  Gunakan `tf.GradientTape` untuk menghitung *loss*.
3.  Hitung gradien menggunakan tape.
4.  [cite_start]Terapkan gradien ke optimizer menggunakan `apply_gradients()`[cite: 2635, 2639, 2653].

In [ ]:
# Contoh kerangka Custom Training Loop
# (Pastikan model dan optimizer sudah didefinisikan sebelumnya)

# for epoch in range(1, n_epochs + 1):
#     print(f"Epoch {epoch}/{n_epochs}")
#     for step, (X_batch, y_batch) in enumerate(train_set):
#         with tf.GradientTape() as tape:
#             y_pred = model(X_batch, training=True)
#             loss = loss_fn(y_batch, y_pred)
#         
#         gradients = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## **9. TensorFlow Functions dan Graphs**

TensorFlow 2.0 menggunakan mode *eager execution* secara default yang mudah di-debug tetapi kurang optimal. [cite_start]Untuk meningkatkan performa, kita bisa mengubah fungsi Python menjadi **TensorFlow Function** (grafik komputasi) menggunakan dekorator `@tf.function`[cite: 2684, 2696].

[cite_start]Fitur **AutoGraph** akan menganalisis kode Python (termasuk loop `for`, `if`, dll.) dan mengubahnya menjadi operasi grafik TensorFlow yang efisien[cite: 2722].

In [ ]:
@tf.function
def cube(x):
    return x ** 3

print("TF Function result:", cube(tf.constant(2.0)))

## **Kesimpulan**

Bab ini telah membekali Anda dengan kemampuan untuk menyesuaikan hampir setiap aspek dari TensorFlow:
* Menggunakan operasi Tensor tingkat rendah.
* Membuat *Loss*, *Metric*, *Layer*, dan *Model* kustom.
* Memahami *Automatic Differentiation* dengan `GradientTape`.
* Menulis *Training Loop* manual untuk fleksibilitas maksimal.
* Mengoptimalkan performa dengan `@tf.function` dan grafik.

Kemampuan ini sangat penting ketika Anda ingin mengimplementasikan ide-ide penelitian terbaru yang belum tersedia di pustaka standar Keras.